In [22]:
%matplotlib inline
from pathlib import Path
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
import matplotlib.pylab as plt
from dmba import plotDecisionTree, classificationSummary, regressionSummary
import time 
import scipy.stats as stats
from xgboost import XGBClassifier
pd.set_option('display.max_rows', None)

# Build a model to predict graduate on time but cannot use last two semesters worth of grades

In [23]:
# create df and set StudentID as index
student_class_df = pd.read_csv('StudentClass.csv')
#student_class_df.set_index('StudentID', inplace=True)
student_class_df = student_class_df.dropna()

In [3]:
# create dataframe from StudentGraduate and set StudentID as index
student_grad_df = pd.read_csv('StudentGraduate.csv')
student_grad_df.set_index('ID', inplace=True)
student_grad_df = student_grad_df.dropna()

In [18]:
# replace grades with quantitative values
# A = 4, B = 3, C = 2, D =1, F = 0
student_class_df.loc[student_class_df['GRADE'] == 'A', 'GRADE'] = '4'
student_class_df.loc[student_class_df['GRADE'] == 'B', 'GRADE'] = '3'
student_class_df.loc[student_class_df['GRADE'] == 'C', 'GRADE'] = '2'
student_class_df.loc[student_class_df['GRADE'] == 'D', 'GRADE'] = '1'
student_class_df.loc[student_class_df['GRADE'] == 'F', 'GRADE'] = '0'

In [19]:
# convert GRADE to integer as the former datatype will be string
student_class_df['GRADE'] = student_class_df['GRADE'].astype('int')

In [20]:
# convert ClassDate from string to datetime
student_class_df['ClassDate'] = pd.to_datetime(student_class_df['ClassDate'])

In [30]:
# creates semesters from each class
# not entirely sure how this works but I found it on Stackoverflow lol
student_class_df['Semester'] = student_class_df.ClassDate.dt.year.astype(str) + "S" + \
np.where(student_class_df.ClassDate.dt.quarter.gt(2),1,2).astype(str)

# student_class_df.numpy .size(20)


AttributeError: Can only use .dt accessor with datetimelike values

In [27]:
# calculate student gpa by semester
student_grades_by_sem = student_class_df.groupby(['StudentID', 'Semester']).mean('Grade')
student_grades_by_sem

KeyError: 'Semester'

In [9]:
students_sem = student_class_df.groupby(['StudentID', 'Semester']).count()
students_sem.drop(['GRADE', 'ClassWeekDay', 'ClassMeetTime', 'ClassDate', 'ClassTitle'], axis=1, inplace=True)#

In [10]:
# drop the last two semesters for each student 
student_sem_except_last_2 = students_sem.drop(students_sem.groupby('StudentID').tail(2).index, axis=0)

In [11]:
# we have a df now of student ID and their college semesters except for their last two
student_sem_except_last_2

Empty DataFrame
Columns: []
Index: [(1.0, 2004 S1), (1.0, 2005 S1), (2.0, 2005 S1), (2.0, 2006 S1), (3.0, 2004 S1), (3.0, 2004 S2), (4.0, 2005 S1), (4.0, 2005 S2), (4.0, 2006 S1), (4.0, 2006 S2), (5.0, 2006 S1), (5.0, 2006 S2), (5.0, 2007 S2), (6.0, 2006 S1), (6.0, 2006 S2), (6.0, 2007 S1), (7.0, 2004 S1), (7.0, 2005 S1), (8.0, 2004 S1), (8.0, 2006 S1), (8.0, 2006 S2), (8.0, 2007 S1), (9.0, 2004 S1), (9.0, 2004 S2), (9.0, 2005 S1), (10.0, 2009 S1), (10.0, 2010 S1), (10.0, 2010 S2), (11.0, 2006 S1), (11.0, 2006 S2), (12.0, 2004 S1), (12.0, 2004 S2), (12.0, 2006 S1), (12.0, 2006 S2), (12.0, 2007 S1), (13.0, 2004 S1), (13.0, 2005 S1), (13.0, 2005 S2), (13.0, 2006 S1), (13.0, 2006 S2), (14.0, 2005 S1), (15.0, 2004 S1), (15.0, 2004 S2), (16.0, 2010 S1), (16.0, 2010 S2), (17.0, 2010 S1), (17.0, 2010 S2), (17.0, 2011 S1), (18.0, 2004 S1), (18.0, 2004 S2), (18.0, 2005 S1), (18.0, 2005 S2), (19.0, 2004 S1), (19.0, 2005 S1), (19.0, 2005 S2), (20.0, 2004 S1), (20.0, 2004 S2), (20.0, 2005 S1), (20.0, 2005 S2), (21.0, 2009 S1), (21.0, 2010 S1), (21.0, 2010 S2), (21.0, 2011 S1), (22.0, 2005 S1), (22.0, 2006 S1), (22.0, 2006 S2), (23.0, 2004 S1), (23.0, 2004 S2), (23.0, 2005 S1), (24.0, 2007 S1), (24.0, 2008 S1), (24.0, 2008 S2), (25.0, 2004 S1), (25.0, 2004 S2), (26.0, 2004 S1), (26.0, 2004 S2), (27.0, 2005 S1), (27.0, 2006 S1), (27.0, 2006 S2), (27.0, 2007 S1), (28.0, 2004 S1), (28.0, 2005 S1), (28.0, 2005 S2), (29.0, 2004 S1), (29.0, 2004 S2), (29.0, 2005 S1), (30.0, 2004 S1), (30.0, 2005 S1), (30.0, 2005 S2), (31.0, 2004 S1), (31.0, 2005 S1), (31.0, 2005 S2), (32.0, 2004 S1), (32.0, 2004 S2), (33.0, 2006 S1), (33.0, 2006 S2), (34.0, 2004 S1), (34.0, 2004 S2), (34.0, 2005 S1), (35.0, 2004 S1), ...]

In [12]:
# this merges the student's gpa with the student's semesters 
# allows us to see student gpa by semester excluding their last two
student_sem_except_last_2_with_gpa = pd.merge(student_grades_by_sem, student_sem_except_last_2, on=['StudentID', 'Semester'])
student_sem_except_last_2_with_gpa

GRADE
StudentID Semester          
1.0       2004 S1   3.250000
          2005 S1   3.000000
2.0       2005 S1   3.250000
          2006 S1   3.000000
3.0       2004 S1   3.500000
          2004 S2   4.000000
4.0       2005 S1   2.000000
          2005 S2   3.000000
          2006 S1   3.000000
          2006 S2   3.666667
5.0       2006 S1   3.500000
          2006 S2   4.000000
          2007 S2   2.666667
6.0       2006 S1   2.333333
          2006 S2   3.000000
          2007 S1   3.000000
7.0       2004 S1   2.000000
          2005 S1   3.000000
8.0       2004 S1   3.000000
          2006 S1   4.000000
          2006 S2   3.000000
          2007 S1   4.000000
9.0       2004 S1   2.500000
          2004 S2   3.000000
          2005 S1   2.500000
10.0      2009 S1   3.000000
          2010 S1   3.333333
          2010 S2   3.500000
11.0      2006 S1   2.250000
          2006 S2   2.500000
12.0      2004 S1   1.500000
          2004 S2   1.000000
          2006 S1   2.000000
          2006 S2   0.000000
          2007 S1   2.000000
13.0      2004 S1   3.500000
          2005 S1   3.000000
          2005 S2   3.500000
          2006 S1   4.000000
          2006 S2   4.000000
14.0      2005 S1   2.400000
15.0      2004 S1   3.000000
          2004 S2   3.000000
16.0      2010 S1   2.750000
          2010 S2   3.666667
17.0      2010 S1   2.333333
          2010 S2   2.500000
          2011 S1   2.000000
18.0      2004 S1   1.500000
          2004 S2   2.000000
          2005 S1   1.000000
          2005 S2   2.000000
19.0      2004 S1   3.000000
          2005 S1   3.500000
          2005 S2   3.500000
20.0      2004 S1   3.000000
          2004 S2   3.000000
          2005 S1   3.000000
          2005 S2   3.000000
21.0      2009 S1   2.000000
          2010 S1   2.000000
          2010 S2   1.000000
          2011 S1   3.000000
22.0      2005 S1   4.000000
          2006 S1   3.500000
          2006 S2   4.000000
23.0      2004 S1   2.500000
          2004 S2   2.333333
          2005 S1   2.000000
24.0      2007 S1   2.000000
          2008 S1   2.500000
          2008 S2   2.750000
25.0      2004 S1   1.200000
          2004 S2   2.000000
26.0      2004 S1   1.000000
          2004 S2   2.000000
27.0      2005 S1   2.000000
          2006 S1   3.000000
          2006 S2   2.500000
          2007 S1   2.666667
28.0      2004 S1   3.000000
          2005 S1   0.000000
          2005 S2   2.000000
29.0      2004 S1   1.666667
          2004 S2   2.000000
          2005 S1   2.000000
30.0      2004 S1   3.333333
          2005 S1   3.000000
          2005 S2   4.000000
31.0      2004 S1   3.000000
          2005 S1   2.000000
          2005 S2   2.666667
32.0      2004 S1   1.333333
          2004 S2   2.666667
33.0      2006 S1   2.666667
          2006 S2   3.666667
34.0      2004 S1   2.000000
          2004 S2   3.000000
          2005 S1   2.500000
35.0      2004 S1   3.000000
          2005 S1   3.000000
36.0      2006 S1   3.250000
          2006 S2   2.000000
37.0      2004 S1   2.000000
          2005 S1   4.000000
          2005 S2   4.000000
38.0      2004 S1   2.666667
          2005 S1   2.000000
39.0      2004 S1   4.000000
          2004 S2   2.750000
40.0      2004 S1   1.666667
          2004 S2   1.000000
          2005 S1   3.000000
          2005 S2   2.000000
41.0      2004 S1   3.000000
          2005 S1   2.000000
          2005 S2   2.666667
          2006 S1   1.000000
42.0      2004 S1   2.000000
          2005 S1   2.666667
          2005 S2   2.666667
43.0      2005 S1   3.000000
          2005 S2   3.000000
44.0      2004 S1   3.000000
          2004 S2   3.333333
45.0      2004 S1   2.000000
          2004 S2   3.000000
          2005 S1   3.000000
46.0      2004 S1   2.000000
          2004 S2   2.750000
47.0      2004 S1   2.400000
          2004 S2   2.500000
48.0      2006 S1   3.000000
          2006 S2   3.333333
49.0      2004 S1   2.333333
          2005 S1   3.000000
50.0      2004 S1   2

In [17]:
# we can do another groupby to find the student's overall gpa for their program without their last 2 semesters' grades
student_gpa_except_last_2_sem = student_sem_except_last_2_with_gpa.groupby('StudentID').mean('GRADE')
student_gpa_except_last_2_sem

,GRADE
StudentID,
1.0,3.125000
2.0,3.125000
3.0,3.750000
4.0,2.916667
5.0,3.388889
6.0,2.777778
7.0,2.500000
8.0,3.500000
9.0,2.666667


In [14]:
# we merge the student's gpa dataframe from above with our original the student_grad_df
# we now have all relevant attributes including their CollegeGPA which excludes their last two semesters
students_df = pd.merge(student_grad_df, student_gpa_except_last_2_sem, left_index=True, right_index=True)

# drop sex and major as they are not quantitative measures and cannot be used
students_df.drop(['sex', 'Major'], axis=1, inplace=True)
students_df.index.name = 'StudentID'
students_df = students_df.rename(columns={'GRADE': 'CollegeGPA'})
students_df

,HS_ENGLISH,HS_MATH,HS_HISTORY,HS_LAB_SCIENCES,HS_FOREIGN_LANGUAGE,HS_ART,HS_ELECTIVES,HS_GPA,WRITINGScore,GraduateONTime,CollegeGPA
StudentID,,,,,,,,,,,
1.0,8,8,7,8,6,4,1,3.691959,428,0,3.125000
2.0,8,10,7,6,8,10,0,3.712993,558,0,3.125000
3.0,8,7,6,6,4,4,0,4.373719,634,1,3.750000
4.0,8,8,6,6,4,2,0,4.046570,386,0,2.916667
5.0,8,6,6,4,4,3,1,2.560076,381,0,3.388889
6.0,8,8,6,4,6,4,1,3.221351,449,0,2.777778
7.0,8,10,6,8,6,4,3,2.470296,523,0,2.500000
8.0,8,8,8,9,5,2,3,2.602701,542,1,3.500000
9.0,10,8,6,6,6,3,2,3.851940,288,0,2.666667


In [15]:
# set training and testing data
# using test data size of 25%
X = students_df.drop(columns=['GraduateONTime'])
y = students_df['GraduateONTime']
train_X, valid_X, train_y, valid_y = train_test_split(X, y, test_size=0.25, random_state=1)

In [16]:
# using gamma=9 yields the highest results using XGBoost
model = XGBClassifier(gamma=9)
model.fit(train_X, train_y)
classificationSummary(train_y, model.predict(train_X))
classificationSummary(valid_y, model.predict(valid_X))

Confusion Matrix (Accuracy 0.7560)

       Prediction
Actual   0   1
     0 557   0
     1 183  10
Confusion Matrix (Accuracy 0.7560)

       Prediction
Actual   0   1
     0 187   1
     1  60   2


# Conclusion
After removing the last two semesters of classes from each student then calculating then cumulative GPA, we are able to then build a model using XGBoost to try and predict graduation rates. Using XGBoost with gamma=9 yields us 75% prediction rate, which, in the real world, is very high. This model is valid. 